In [1]:
import pandas as pd
import json
import requests
import numpy as np
import openpyxl
import xlsxwriter
from datetime import datetime

In [92]:
# authentication token retrieval
data = {
  "username": "###",
  "password": "###"
}

response = requests.post('https://us.elabjournal.com/api/v1/auth/user', data=data)
response = response.json()
token = response.get('token')

def getSamples(freezer_id):
    """This function submits GET requests to retrieve expired samples from a given freezer"""
    headers = {
        'Accept': 'application/json',
        'X-Requested-With': 'Swagger',
        'Authorization': token  
    }
    page_number = 0
    url_first_half = 'https://us.elabjournal.com/api/v1/samples?$expand=meta&$expand=quantity&$page='
    url_second_half = '&storageID=' + str(freezer_id)
    loop = True
    samples = []
    while loop == True:
        url = url_first_half + str(page_number) + url_second_half
        page_of_samples = requests.get(url, headers=headers)
        page_of_samples = page_of_samples.json()
        print('On Page', page_number)
        if page_of_samples.get('recordCount') == 0:
            loop = False
            break
        page_of_samples = page_of_samples.get('data')
        page_number += 1
        samples = samples + page_of_samples
    return samples

def sampleFormatting(sample):
    meta = (sample.get('meta'))
    for custom in meta:
        if custom['key'] == 'Concentration (mg/mL)':
            sample['Concentration (mg/mL)'] = custom['value']
        if custom['key'] == 'LN or NB reference':
            sample['LN or NB reference'] = custom['value']
        if custom['key'] == 'Part Number':
            sample['Part Number'] = custom['value']
    sample.pop('archived')
    sample.pop('meta')
    sample.pop('userID')
    sample.pop('storageLayerID')
    sample.pop('barcode')
    sample.pop('sampleType')
    sample.pop('sampleTypeID')
    sample.pop('checkedOut')
    sample.pop('parentSampleID')
    sample.pop('creatorID')
    sample.pop('sampleID')
    return sample

def excelWrite(expired_samples, samples_length):
    expired_samples = pd.DataFrame(expired_samples)
    print('Checked through', samples_length, 'samples')
    print('There are', len(expired_samples), 'expired samples in', freezer_name)
    excel_sheet = 'expired samples in ' + freezer_name  + '.xlsx'
    expired_samples.to_excel(excel_sheet)
    print("Excel sheet named'", excel_sheet, "'saved to local directory")

def retrieve(freezer_name, freezer_id):
    samples = getSamples(freezer_id)
    expired_samples = []
    for sample in samples:
        expiration = sample.get('expiration')
        if expiration != None:
            now = datetime.now()
            current_date = str(now.date())
            if expiration < current_date:
                meta = (sample.get('meta'))
                sample = sampleFormatting(sample)
                expired_samples.append(sample)
    excelWrite(expired_samples, len(samples))

# MFG FREEZER 148

In [93]:
freezer_name = 'MFG Freezer 148'
freezer_id = 69875
retrieve(freezer_name, freezer_id)


On Page 0
On Page 1
On Page 2
Checked through 1013 samples
There are 40 expired samples in MFG Freezer 148
Excel sheet named' expired samples in MFG Freezer 148.xlsx 'saved to local directory


# MFG FREEZERS

In [94]:
freezer_name = 'MFG Freezer 163'
freezer_id = 85914
retrieve(freezer_name, freezer_id)

freezer_name = 'MFG Freezer 148'
freezer_id = 69875
retrieve(freezer_name, freezer_id)

freezer_name = 'MFG Freezer 33'
freezer_id = 38046
retrieve(freezer_name, freezer_id)

freezer_name = 'MFG Refrigerator 106'
freezer_id = 39168
retrieve(freezer_name, freezer_id)

freezer_name = "Purification Bay 300.7L"
freezer_id = 49378
retrieve(freezer_name, freezer_id)

freezer_name = "Clinical Sample #153"
freezer_id = 56455
retrieve(freezer_name, freezer_id)


On Page 0
On Page 1
On Page 2
Checked through 1530 samples
There are 56 expired samples in MFG Freezer 163
Excel sheet named' expired samples in MFG Freezer 163.xlsx 'saved to local directory
On Page 0
On Page 1
On Page 2
Checked through 1013 samples
There are 40 expired samples in MFG Freezer 148
Excel sheet named' expired samples in MFG Freezer 148.xlsx 'saved to local directory
On Page 0
On Page 1
Checked through 14 samples
There are 5 expired samples in MFG Freezer 33
Excel sheet named' expired samples in MFG Freezer 33.xlsx 'saved to local directory
On Page 0
On Page 1
Checked through 31 samples
There are 29 expired samples in MFG Refrigerator 106
Excel sheet named' expired samples in MFG Refrigerator 106.xlsx 'saved to local directory
On Page 0
On Page 1
Checked through 1 samples
There are 0 expired samples in Purification Bay 300.7L
Excel sheet named' expired samples in Purification Bay 300.7L.xlsx 'saved to local directory
On Page 0
On Page 1
On Page 2
On Page 3
On Page 4
On Pa

# R&D FREEZERS

In [5]:
freezer_name = 'R&D Freezer 134'
freezer_id = 38043
retrieve(freezer_name, freezer_id)


freezer_name = 'R&D Freezer 155'
freezer_id = 59896
retrieve(freezer_name, freezer_id)

freezer_name = 'R&D Freezer 154'
freezer_id = 59899
retrieve(freezer_name, freezer_id)


On Page 0
On Page 1
Checked through 402 samples
There are 0 expired samples in R&D Freezer 134
Excel sheet named' expired samples in R&D Freezer 134.xlsx 'saved to local directory
On Page 0
On Page 1
On Page 2
On Page 3
On Page 4
Checked through 3432 samples
There are 18 expired samples in R&D Freezer 155
Excel sheet named' expired samples in R&D Freezer 155.xlsx 'saved to local directory


# TEST ALL

In [6]:
freezer_name = 'All'
freezer_id = ''
retrieve(freezer_name, freezer_id)

On Page 0
On Page 1
On Page 2
On Page 3
On Page 4
On Page 5
On Page 6
On Page 7
On Page 8
On Page 9
On Page 10
On Page 11
On Page 12
On Page 13
On Page 14
On Page 15
On Page 16
On Page 17
On Page 18
On Page 19
On Page 20
On Page 21
On Page 22
On Page 23
On Page 24
On Page 25
On Page 26
On Page 27
On Page 28
On Page 29
On Page 30
On Page 31
On Page 32
On Page 33
On Page 34
On Page 35
On Page 36
On Page 37
On Page 38
On Page 39
On Page 40
On Page 41
On Page 42
On Page 43
On Page 44
On Page 45
On Page 46
On Page 47
On Page 48
On Page 49
On Page 50
On Page 51
On Page 52
On Page 53
On Page 54
On Page 55
On Page 56
On Page 57
On Page 58
On Page 59
On Page 60
On Page 61
On Page 62
On Page 63
On Page 64
Checked through 63320 samples
There are 636 expired samples in All
Excel sheet named' expired samples in All.xlsx 'saved to local directory
